# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [11]:
## IMPORTS
# core config
from azureml.core import Workspace, Experiment
# compute config
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#HD config

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.policy import TruncationSelectionPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'EXP_HEART_FAILURE_HD'

experiment=Experiment(ws, experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-145450
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-145450


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.
I am planning to use xgboost model for the hyperparameter tuning.
I have done a baseline modeling on 2 different model types, locally
1- Random Forest, which is very simple, well interpretable ensemble model, hard/impossible to overtrain, works well on non-linear problems
2- xgboost, which is a boosted ensemble model, with a lots of tunable parameters

With RF I was able to get a 0.73 AUC, which is a medium strength model.
With xgboost, using 5 fold CV, but without really running through parameter space locally I was able to achieve 0.88
The catch with the xgboost, that it typically performs well on large dataset, the provided dataset has 300 rows only, so there is a risk of overtraining
So I will need to focus on regularization of the model.

I am using 2 paths for this: an indirect regularization with boost rounds and learning rate and a direct one with gamma and lambdta


Target metric, as the dataset is for classification problem is AUC.

Initially I am using RandomParameterSampling to run through a wider parameter space. 
For this I will use an aggressive erly stopping policy, to save time and money.

As I am using xgboost model, I am not using estimator, I use a run config instead, with an xgboost model

From the learnings I will set up another round 


In [5]:
#As i do not want to wait for the cluster nodes to get activated, I set up all of them active immediately
cpu_cluster_name = "hd-gyula-01"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("This compute exists")
except ComputeTargetException:
    print("Building my own")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=4, max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Building my own
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [26]:
# Initial parameter space with sampler, many parameters, wide space
# https://xgboost.readthedocs.io/en/latest/parameter.html
ps_random = {"n_estimators": choice(50, 100, 150, 200, 300, 500),
              "max_depth": choice(3, 5, 8, 13),
              "min_samples_split": choice(2,3,4)
              }

random_samp = RandomParameterSampling(parameter_space=ps_random)

In [20]:
truncation_selection_policy = TruncationSelectionPolicy(evaluation_interval=1, truncation_percentage=30, delay_evaluation=5)

In [25]:
# instead of estimator i am using a run config
src = ScriptRunConfig(source_directory="./script",
                      script="hyper_tuning_rf.py",
                      arguments=['--n_estimators', 10, '--max_depth', 3, '--min_samples_split', 2],
                      compute_target=compute_target)

In [23]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = truncation_selection_policy

#TODO: Create the different params that you will be using during training
param_sampling = random_samp

#TODO: Create your estimator and hyperdrive config
#estimator = <your estimator here>

hyperdrive_run_config =  HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name="AUC",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [12]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [27]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [28]:
widget1 = RunDetails(hyperdrive_run)
widget1.show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# display properties of the model

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service